**About** : This notebook is used to train RNN models.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ../src/

/home/tviel/work/kaggle_rsna_lumbar_spine/src


/home/tviel/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


### Imports

In [3]:
import os
import sys
import glob
import json
import torch
import operator
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from sklearn.metrics import *
from collections import Counter
from scipy.stats import spearmanr

warnings.simplefilter(action="ignore", category=UserWarning)

In [4]:
from util.logger import (
    prepare_log_folder,
    save_config,
    create_logger,
)

from data.dataset import FeatureDataset
from params import *
from data.preparation import *
from util.logger import Config as ConfigInf
from training.main_lvl2 import k_fold
from util.metrics import *

In [5]:
from model_zoo.models_lvl2 import define_model
from training.losses import StudyLoss
from util.metrics import rsna_loss

### Data

In [6]:
df = prepare_data_lvl2()

if "fold" not in df.columns:
    folds = pd.read_csv("../input/train_folded_v1.csv")
    df = df.merge(folds, how="left")

In [7]:
df = df[df['fold'] == 0].reset_index(drop=True)

In [8]:
EXP_FOLDERS = {
    # "scs_crop": "../logs/2024-08-29/15/",  # 15
    # "nfn_crop": "../logs/2024-08-29/16/",  # 30
    "scs_crop_coords":  "../logs/2024-08-29/17/",
    "nfn_crop_coords":  "../logs/2024-08-29/18/",

    # "dh": '../output/oof____cfg_dh_12s4c.pth',  # Darragh preds
    # "ch": '../output/oof_cfg_ch_35.pth',  # Dieter preds
    "crop": "../logs/2024-08-29/5/",  # 75
}

In [9]:
config = ConfigInf(json.load(open(EXP_FOLDERS["crop"] + "config.json", "r")))

In [10]:
df.head(1)

,study_id,series_id,series_description,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,spinal_canal_stenosis_l4_l5,spinal_canal_stenosis_l5_s1,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,...,left_subarticular_stenosis_l2_l3,left_subarticular_stenosis_l3_l4,left_subarticular_stenosis_l4_l5,left_subarticular_stenosis_l5_s1,right_subarticular_stenosis_l1_l2,right_subarticular_stenosis_l2_l3,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1,fold
0,7143189,"[132939515, 1951927562, 3219733239]","[Sagittal T2/STIR, Axial T2, Sagittal T1]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
dataset = FeatureDataset(df, EXP_FOLDERS, targets=CLASSES)

In [12]:
# p = []
# for i in tqdm(range(len(dataset))):
#     p_ = dataset[i][0]["nfn_crop_coords"].view(-1, 3).numpy()
#     # p_ = p_.reshape(5, 2, 3).transpose(1, 0, 2).reshape(10, 3)
#     p.append(p_)
# p = np.array(p)

# y = df[df.columns[8:18]].values

# aucs = []
# for i, c in enumerate(CLASSES[5:15]):
#     auc = disk_auc(y[:, i], p[:, i])
#     print(f'{c} AUC: \t {auc :.4f}')
#     aucs.append(auc)
# print(f'\n-> Avg AUC: \t {np.mean(aucs) :.4f}')

In [13]:
for i in tqdm(range(len(dataset))):
    fts, y, _ = dataset[i]
    # for k in fts:
    #     print(k, fts[k].size())
    # break

  0%|          | 0/494 [00:00<?, ?it/s]

In [14]:
idx = 2
fts, y, _ = dataset[idx]

In [15]:
for k in fts:
    print(k, fts[k].size())

scs_crop_coords torch.Size([5, 3])
nfn_crop_coords torch.Size([10, 3])
crop torch.Size([25, 6])


In [16]:
# plt.figure(figsize=(20, 5))
# plt.subplot(1, 3, 1)
# for i in range(5):
#     plt.plot(fts['ss_aux'].softmax(1)[:, i], label=LEVELS[i])
# plt.legend()

# plt.subplot(1, 3, 2)
# for i in [1, 2]:
#     plt.plot(fts['ss'][:, i], label=f'left_{i}')
# for i in [4, 5]:
#     plt.plot(fts['ss'][:, i], label=f'right_{i - 3}')
# plt.legend()

# # plt.subplot(1, 3, 3)
# # for i in range(5):
# #     plt.plot(fts['ss'][:, 1, i], label=LEVELS[i])
# # plt.legend()

# plt.show()

In [17]:
# for k in fts:
#     print(k, fts[k].shape)

### Model

In [18]:
# model = define_model(
#     name="simple",
#     num_classes=len(CLASSES) * 3,
#     layer_dim=0,
#     ft_dim=64,
#     n_fts=45 + 75,
#     resize=10,
# )

In [19]:
x = {k: fts[k].unsqueeze(0) for k in fts}

In [20]:
# pred, _ = model(x)
# pred.size()

In [21]:
# l = StudyLoss()
# l(pred, y.unsqueeze(0))

In [22]:
# rsna_loss(y.unsqueeze(0).numpy(), pred.softmax(2).detach().numpy())

### Training

In [23]:
class Config:
    """
    Parameters used for training
    """
    # General
    seed = 42
    verbose = 1
    device = "cuda"
    save_weights = True
    targets = CLASSES

    # Data
    exp_folders = {
        # "scs_crop": "../logs/2024-08-29/15/",  # 15
        # "nfn_crop": "../logs/2024-08-29/16/",  # 30
        # "scs_crop_coords":  "../logs/2024-08-29/17/",
        # "nfn_crop_coords":  "../logs/2024-08-29/18/",

        # "dh": '../output/oof____cfg_dh_12s4c.pth',  # Darragh preds
        # "ch": '../output/oof_cfg_ch_35.pth',  # Dieter preds
        "crop": "../logs/2024-08-29/5/",  # 75

        # "nfn_crop_coords": "../logs/2024-09-02/27/",  # Fix coords crop coords
        # "nfn_crop_coordsax": "../logs/2024-09-03/2/",   # Fix coords crop coords ax
        # "scs_crop_coords": "../logs/2024-08-30/6/",  # spinenet - 15 
        # "nfn_crop_coords": "../logs/2024-08-30/7/",  # spinenet - 30

        
    }
    n_fts = 0
    resize = 0
    remove_noisy = False

    # k-fold
    k = 4
    folds_file = "../input/train_folded_v1.csv"  # f"../input/folds_{k}.csv"
    selected_folds = [0, 1, 2, 3]

    # Model
    name = "simple"
    dense_dim = 4096
    layer_dim = 0
    ft_dim = [6, 6, 6]  # scs, nfn, ss

    p = 0.
    num_classes = len(CLASSES) * 3
    num_classes_aux = 0

    # Training    
    loss_config = {
        "name": "study",
        "weighted": True,
        "use_any": True,
        "smoothing": 0,
        "activation": "study",
        "aux_loss_weight": 0,
        "name_aux": "",
        "smoothing_aux": 0,
        "activation_aux": "",
    }

    data_config = {
        "batch_size": 128,
        "val_bs": 512,
        "mix": "mixup",
        "mix_proba": 0.,
        "sched": False,
        "mix_alpha": 4.,
        "additive_mix": False,
        "num_classes": num_classes,
        "num_classes_aux": num_classes_aux,
        "num_workers": 8,
    }

    optimizer_config = {
        "name": "AdamW",
        "lr": 5e-5,
        "warmup_prop": 0.,
        "betas": (0.9, 0.999),
        "max_grad_norm": 1.,
        "weight_decay": 1,
    }

    epochs = 15

    use_fp16 = True
    verbose = 1
    verbose_eval = 20

    fullfit = False
    n_fullfit = 1

    local_rank = 0
    distributed = False
    world_size = 1

In [24]:
DEBUG = False
log_folder = None

In [25]:
df = prepare_data_lvl2()

if "fold" not in df.columns:
    folds = pd.read_csv(Config.folds_file)
    df = df.merge(folds, how="left")

# df = df[~df['study_id'].isin([1215498865, 1647904243, 2570933394, 2761048584, 3284652867, 3941522676])].reset_index(drop=True)

In [26]:
# if not DEBUG:
#     log_folder = prepare_log_folder(LOG_PATH)
#     print(f"Logging results to {log_folder}")
#     config_df = save_config(Config, log_folder + "config.json")
#     create_logger(directory=log_folder, name="logs.txt")

# preds = k_fold(Config, df, log_folder=log_folder, run=None)

### Eval

In [27]:
avg_loss, losses = rsna_loss(df[Config.targets].values, preds)

for k, v in losses.items():
    print(f"- {k}_loss\t: {v:.3f}")

print(f'\n -> CV Score : {avg_loss :.4f}')

NameError: name 'preds' is not defined

In [28]:
avg_loss, losses = rsna_loss(df[Config.targets].values, preds)

for k, v in losses.items():
    print(f"- {k}_loss\t: {v:.3f}")

print(f'\n -> CV Score : {avg_loss :.4f}')

NameError: name 'preds' is not defined

In [ ]:
avg_loss, losses = rsna_loss(df[Config.targets].values, preds)

for k, v in losses.items():
    print(f"- {k}_loss\t: {v:.3f}")

print(f'\n -> CV Score : {avg_loss :.4f}')

- scs_loss	: 0.272
- nfn_loss	: 0.487
- ss_loss	: 0.586
- any_loss	: 0.268

 -> CV Score : 0.4032


In [ ]:
avg_loss, losses = rsna_loss(df[Config.targets].values, preds)

for k, v in losses.items():
    print(f"- {k}_loss\t: {v:.3f}")

print(f'\n -> CV Score : {avg_loss :.4f}')

- scs_loss	: 0.286
- nfn_loss	: 0.488
- ss_loss	: 0.585
- any_loss	: 0.284

 -> CV Score : 0.4107


In [208]:
# yy = pkl["target"].contiguous()[order].cpu().numpy().clip(-1, 2)
# for i in range(len(df)):
#     if not (df[Config.targets].values[i] == yy[i]).all():
#         print(i, yy[i].tolist(), df[Config.targets].values[i].tolist())
#         display(df.iloc[[i]])
#         # break

In [33]:
pkl = torch.load('../output/oof_cfg_ch_35.pth')
pkl = torch.load('../output/oof____cfg_dh_12s1.pth')

df = df[~df['study_id'].isin([1215498865, 1647904243, 2570933394, 2761048584, 3284652867, 3941522676])].reset_index(drop=True)

order = [pkl['study_id'].tolist().index(s) for s in df['study_id'].values]

avg_loss, losses = rsna_loss(df[Config.targets].values, pkl["logits"].cpu().float().softmax(-1).numpy()[order])

for k, v in losses.items():
    print(f"- {k}_loss\t: {v:.3f}")

print(f'\n -> CV Score : {avg_loss :.4f}')

- scs_loss	: 0.319
- nfn_loss	: 0.504
- ss_loss	: 1.099
- any_loss	: 0.309

 -> CV Score : 0.5576


In [34]:
# aucs = []
# for i, c in enumerate(CLASSES):
#     auc = disk_auc(df[Config.targets].values[:, i], preds[:, i])
#     print(f'{c} AUC: \t {auc :.4f}')
#     aucs.append(auc)
# print(f'\n-> Avg AUC: \t {np.mean(aucs) :.4f}')

In [37]:
# aucs = []
# for i, c in enumerate(CLASSES):
#     auc = disk_auc(df[Config.targets].values[:, i], preds_dd[:, i])
#     print(f'{c} AUC: \t {auc :.4f}')
#     aucs.append(auc)
# print(f'\n-> Avg AUC: \t {np.mean(aucs) :.4f}')

In [301]:
# preds = preds[df['fold'].values == 0]
# df = df[df['fold'].values == 0]

In [268]:
EXP_FOLDER = "../logs/2024-09-02/27/"

p = np.load(EXP_FOLDER + "pred_inf_0.npy").reshape(-1, 10, 3)
df_val = pd.read_csv(EXP_FOLDER + "df_val_0.csv")

y = df_val['target'].values.reshape(-1, 10)

aucs = []
for i, c in enumerate(CLASSES[5:15]):
    auc = disk_auc(y[:, i], p[:, i])
    print(f'{c} AUC: \t {auc :.4f}')
    aucs.append(auc)
print(f'\n-> Avg AUC: \t {np.mean(aucs) :.4f}')

left_neural_foraminal_narrowing_l1_l2 AUC: 	 0.9613
left_neural_foraminal_narrowing_l2_l3 AUC: 	 0.9558
left_neural_foraminal_narrowing_l3_l4 AUC: 	 0.9164
left_neural_foraminal_narrowing_l4_l5 AUC: 	 0.8718
left_neural_foraminal_narrowing_l5_s1 AUC: 	 0.8831
right_neural_foraminal_narrowing_l1_l2 AUC: 	 0.9699
right_neural_foraminal_narrowing_l2_l3 AUC: 	 0.9284
right_neural_foraminal_narrowing_l3_l4 AUC: 	 0.9227
right_neural_foraminal_narrowing_l4_l5 AUC: 	 0.8932
right_neural_foraminal_narrowing_l5_s1 AUC: 	 0.8843

-> Avg AUC: 	 0.9187


In [173]:
p = preds.reshape(preds.shape[0], 5, 5, 3)
p = p.transpose(0, 2, 1, 3)
p = p.reshape(-1, 5, 3)

y = df[Config.targets].values
y = y.reshape(preds.shape[0], 5, 5)
y = y.transpose(0, 2, 1)
y = y.reshape(-1, 5)

aucs = []
for i, c in enumerate(CLASSES_CROP):
    auc = disk_auc(y[:, i], p[:, i])
    print(f'{c} AUC: \t {auc :.4f}')
    aucs.append(auc)
print(f'\n-> Avg AUC: \t {np.mean(aucs) :.4f}')

spinal_canal_stenosis AUC: 	 0.5000
left_neural_foraminal_narrowing AUC: 	 0.8108
right_neural_foraminal_narrowing AUC: 	 0.8104
left_subarticular_stenosis AUC: 	 0.5000
right_subarticular_stenosis AUC: 	 0.5000

-> Avg AUC: 	 0.6243


In [35]:
for i, c in enumerate(LEVELS_):
    print(c, disk_auc(y[:, i], p[:, i]))

l1_l2 0.8424656813282588
l2_l3 0.9118449814640606
l3_l4 0.9172478409959582
l4_l5 0.8230807887811995
l5_s1 0.8279761385800729


In [49]:
preds[:10]

array([[9.966e-01, 3.572e-03, 2.110e-05],
       [9.941e-01, 5.920e-03, 1.913e-05],
       [9.526e-01, 4.715e-02, 3.810e-04],
       [9.106e-01, 8.887e-02, 4.816e-04],
       [9.756e-01, 2.380e-02, 5.293e-04],
       [9.937e-01, 6.454e-03, 2.307e-05],
       [9.761e-01, 2.408e-02, 8.440e-05],
       [8.501e-01, 1.493e-01, 6.313e-04],
       [8.105e-01, 1.884e-01, 1.161e-03],
       [9.399e-01, 5.789e-02, 2.064e-03]], dtype=float16)

array([[9.966e-01, 3.572e-03, 2.110e-05],
       [9.937e-01, 6.454e-03, 2.307e-05]], dtype=float16)

In [40]:
df_val.head(20)

,study_id,series_id,series_description,orient,weighting,img_path,condition,level,coords,side,target,fold
0,7143189,3219733239,Sagittal T1,Sagittal,T1,../input/coords_crops_0.1_f/7143189_3219733239...,Left Neural Foraminal Narrowing,L1/L2,[ 11. 210.30305116 104.76639614],Left,0,0
1,7143189,3219733239,Sagittal T1,Sagittal,T1,../input/coords_crops_0.1_f/7143189_3219733239...,Left Neural Foraminal Narrowing,L2/L3,[ 11. 205.97976234 147.42098804],Left,0,0
2,7143189,3219733239,Sagittal T1,Sagittal,T1,../input/coords_crops_0.1_f/7143189_3219733239...,Left Neural Foraminal Narrowing,L3/L4,[ 12. 197.02351562 195.78472031],Left,0,0
3,7143189,3219733239,Sagittal T1,Sagittal,T1,../input/coords_crops_0.1_f/7143189_3219733239...,Left Neural Foraminal Narrowing,L4/L5,[ 11. 193.91273442 233.74949645],Left,0,0
4,7143189,3219733239,Sagittal T1,Sagittal,T1,../input/coords_crops_0.1_f/7143189_3219733239...,Left Neural Foraminal Narrowing,L5/S1,[ 11. 199.49312761 272.41812149],Left,0,0
5,7143189,3219733239,Sagittal T1,Sagittal,T1,../input/coords_crops_0.1_f/7143189_3219733239...,Right Neural Foraminal Narrowing,L1/L2,[ 4. 214.34299517 105.73913043],Right,0,0
6,7143189,3219733239,Sagittal T1,Sagittal,T1,../input/coords_crops_0.1_f/7143189_3219733239...,Right Neural Foraminal Narrowing,L2/L3,[ 4. 202.5942029 142.84057971],Right,0,0
7,7143189,3219733239,Sagittal T1,Sagittal,T1,../input/coords_crops_0.1_f/7143189_3219733239...,Right Neural Foraminal Narrowing,L3/L4,[ 4. 200.73913043 187.98067633],Right,0,0
8,7143189,3219733239,Sagittal T1,Sagittal,T1,../input/coords_crops_0.1_f/7143189_3219733239...,Right Neural Foraminal Narrowing,L4/L5,[ 4. 197.02898551 224.46376812],Right,0,0
9,7143189,3219733239,Sagittal T1,Sagittal,T1,../input/coords_crops_0.1_f/7143189_3219733239...,Right Neural Foraminal Narrowing,L5/S1,[ 3. 198.26570048 268.98550725],Right,0,0


- scs_loss	: 0.299
- nfn_loss	: 0.500
- ss_loss	: 0.593
- any_loss	: 0.291

 -> CV Score : 0.4206

In [108]:
for i in [5, 6, 7, 8, 9, 15, 16, 17, 18, 19]:
    j = i + 5
    c = CLASSES[i]
    c2 = CLASSES[j]
    
    ref_auc = disk_auc(df[Config.targets].values[:, i], preds[:, i])
    auc = disk_auc(df[Config.targets].values[:, i], preds[:, j])
    r = (
        spearmanr(preds[:, i][:, 1], preds[:, j][:, 1]).statistic + 
        spearmanr(preds[:, i][:, 2], preds[:, j][:, 2]).statistic
    ) / 2
    print(f'\npred: {c2} \t truth: {c}')
    # print(r)
    y1 = df[Config.targets].values[:, i]
    y2 = df[Config.targets].values[:, j]
    
    eq = (y1[(y1 > 0) & (y2 > 0)] == y2[(y1 > 0) & (y2 > 0)]).mean()
    print(f'Ref AUC          : {ref_auc :.4f}')
    print(f'Swap AUC         : {auc :.4f}')
    print(f'Equal proportion : {eq:.3f}')
    print(f'Preds correlation: {r:.3f}')


# for j in [5, 6, 7, 8, 9, 15, 16, 17, 18, 19]:
#     i = j + 5
#     c = CLASSES[i]
#     c2 = CLASSES[j]
    
#     ref_auc = disk_auc(df[Config.targets].values[:, i], preds[:, i])
#     auc = disk_auc(df[Config.targets].values[:, i], preds[:, j])
#     r = (
#         spearmanr(preds[:, i][:, 1], preds[:, j][:, 1]).statistic + 
#         spearmanr(preds[:, i][:, 2], preds[:, j][:, 2]).statistic
#     ) / 2
#     print(f'\npred: {c2} \t truth: {c}')
#     # print(r)
#     y1 = df[Config.targets].values[:, i]
#     y2 = df[Config.targets].values[:, j]
    
#     eq = (y1[(y1 > 0) & (y2 > 0)] == y2[(y1 > 0) & (y2 > 0)]).mean()
#     print(f'Ref AUC          : {ref_auc :.4f}')
#     print(f'Swap AUC         : {auc :.4f}')
#     print(f'Equal proportion : {eq:.3f}')
#     print(f'Preds correlation: {r:.3f}')


pred: right_neural_foraminal_narrowing_l1_l2 	 truth: left_neural_foraminal_narrowing_l1_l2
Ref AUC          : 0.8969
Swap AUC         : 0.8969
Equal proportion : 0.750
Preds correlation: 1.000

pred: right_neural_foraminal_narrowing_l2_l3 	 truth: left_neural_foraminal_narrowing_l2_l3
Ref AUC          : 0.9309
Swap AUC         : 0.8918
Equal proportion : 0.852
Preds correlation: 0.825

pred: right_neural_foraminal_narrowing_l3_l4 	 truth: left_neural_foraminal_narrowing_l3_l4
Ref AUC          : 0.8844
Swap AUC         : 0.8447
Equal proportion : 0.832
Preds correlation: 0.863

pred: right_neural_foraminal_narrowing_l4_l5 	 truth: left_neural_foraminal_narrowing_l4_l5
Ref AUC          : 0.8647
Swap AUC         : 0.7751
Equal proportion : 0.712
Preds correlation: 0.734

pred: right_neural_foraminal_narrowing_l5_s1 	 truth: left_neural_foraminal_narrowing_l5_s1
Ref AUC          : 0.8633
Swap AUC         : 0.7891
Equal proportion : 0.636
Preds correlation: 0.760

pred: right_subarticular

In [ ]:
losses = []
y = df[Config.targets].values
for i in tqdm(range(len(df))):
    l = rsna_loss(y[i:i+1], preds[i:i+1])[1]
    l.update({"study": df["study_id"].values[i]})
    losses.append(l)
losses = pd.DataFrame(losses)

In [ ]:
plt.figure(figsize=(20, 5))
for i, c in enumerate(losses.columns[:-1]):
    plt.subplot(1, 4, i + 1)
    sns.histplot(losses[c].values)
    plt.title(c)
plt.show()

In [ ]:
losses[losses["scs"] > 2]

In [ ]:
df[df["study_id"] == 1972129014]

In [ ]:
losses[losses["any"] > 2]

- scs_loss	: 0.325
- nfn_loss	: 0.517
- ss_loss	: 0.634
- any_loss	: 0.297

 -> CV Score : 0.443

Done ! 